In [34]:
import json
import pandas as pd

# Carica il file JSONL in un DataFrame
with open('dataset.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

df = pd.DataFrame(data)


In [35]:
#Indico le supercategorie di interesse

g_keys = ["Opere da cartongessista", 
          "Opere di rivestimento", 
          "Opere di pavimentazione", 
          "Opere da serramentista", 
          "Controsoffitti", 
          "Opere da falegname",
          "Apparecchi sanitari e accessori"]

In [36]:
#Creazione maschera e dataframe limitato

mask = df["super"].isin(g_keys)
df_lim = df[mask]

In [37]:
#Normalizzazione testo
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)
    s =s.strip()
    s = " ".join(s.split())

    return s
df_lim = df_lim.copy()
df_lim["text_norm"] = df_lim["text"].map(normalize_text)

In [38]:
#Dedup
#Creazione colonna lunghezza testo e ordinamento
def dedup_prefix_group(g: pd.DataFrame, text_col) -> pd.DataFrame:
    g = g.copy()
    g["_len"] = g[text_col].astype(str).str.len()
    g = g.sort_values("_len", ascending = False)

    keep_idx = []
    keep_texts = []
    for idx, t in g[text_col].items():
        if any(k.startswith(t) for k in keep_texts):
            continue
        keep_idx.append(idx)
        keep_texts.append(t)
    
    out= g.loc[keep_idx].drop(columns=["_len"])
    return out.sort_index()

group_cols = ["super", "cat"]

df_clean = (df_lim
            .groupby(group_cols, group_keys = False)
            .apply(lambda g: dedup_prefix_group(g, text_col="text_norm"))
            .drop(columns=["text_norm"])
            .reset_index(drop=True)
)

C:\Users\f.biggi\AppData\Local\Temp\ipykernel_6704\2174041924.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: dedup_prefix_group(g, text_col="text_norm"))


In [39]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3192 non-null   object
 1   super   3192 non-null   object
 2   cat     3192 non-null   object
dtypes: object(3)
memory usage: 74.9+ KB


In [40]:
print(df_clean["cat"].unique())

["Accessori per l'allestimento di servizi igienici" 'Apparecchi sanitari'
 'Cassette di scarico' "Botole d'ispezione e accessori"
 'Controsoffitti a Baffles e ispezionabili'
 'Controsoffitti a doghe in legno'
 'Controsoffitti in PVC o materiali plastici'
 'Controsoffitti in altri materiali' 'Controsoffitti in cartongesso'
 'Controsoffitti in fibre minerali e acustici' 'Controsoffitti metallici'
 'Velette di raccordo' 'Accessori per cartongessi'
 'Contropareti in cartongesso resistente al fuoco'
 'Contropareti in cartongesso standard e idrorepellente'
 'Contropareti in lastre di fibrocemento'
 'Pareti in cartongesso resistente al fuoco'
 'Pareti in cartongesso standard e idrorepellente'
 'Pareti in lastre di fibrocemento'
 'Setto autoportante cartongesso resistente al fuoco'
 'Setto autoportante in cartongesso standard e idrorepellente'
 'Boiserie in legno' 'Opere in legno custom' 'Persiane e scuri in legno'
 'Porte in legno' 'Avvolgibili, controtelai, cassonetti e persiane'
 'Porte bli

In [41]:
#Rimozione categorie non volute
bad = ["Noli", "Scavi e trasporti a discarica"]
mask = ~df_clean["cat"].isin(bad)
df_clean = df_clean[mask]
print(df_clean["cat"].unique())

["Accessori per l'allestimento di servizi igienici" 'Apparecchi sanitari'
 'Cassette di scarico' "Botole d'ispezione e accessori"
 'Controsoffitti a Baffles e ispezionabili'
 'Controsoffitti a doghe in legno'
 'Controsoffitti in PVC o materiali plastici'
 'Controsoffitti in altri materiali' 'Controsoffitti in cartongesso'
 'Controsoffitti in fibre minerali e acustici' 'Controsoffitti metallici'
 'Velette di raccordo' 'Accessori per cartongessi'
 'Contropareti in cartongesso resistente al fuoco'
 'Contropareti in cartongesso standard e idrorepellente'
 'Contropareti in lastre di fibrocemento'
 'Pareti in cartongesso resistente al fuoco'
 'Pareti in cartongesso standard e idrorepellente'
 'Pareti in lastre di fibrocemento'
 'Setto autoportante cartongesso resistente al fuoco'
 'Setto autoportante in cartongesso standard e idrorepellente'
 'Boiserie in legno' 'Opere in legno custom' 'Persiane e scuri in legno'
 'Porte in legno' 'Avvolgibili, controtelai, cassonetti e persiane'
 'Porte bli

In [42]:
df_clean.to_json(
    "dataset_lim.jsonl",
    lines = True,
    force_ascii = False,
    orient = "records",
    index = False
)